In [86]:
from sage.combinat.permutation import Permutation, Permutations
from sage.matrix.constructor import matrix, block_matrix
from sage.rings.rational_field import QQ
from sage.combinat.symmetric_group_representations import SymmetricGroupRepresentation_generic_class


# ====================================================================
# Induced representation from S_k to S_n
# ====================================================================

class InducedRepresentation(SymmetricGroupRepresentation_generic_class):
    r"""
    Induced representation from `S_k` to `S_n`.

    The subgroup `S_k` acts on `\{1,...,k\}` and fixes `\{k+1,...,n\}`.

    EXAMPLES::

        sage: rep_S2 = SymmetricGroupRepresentation([2])
        sage: ind = InducedRepresentation(3, 2, rep_S2)
        sage: ind.dimension()
        3

        sage: g = Permutation([2,1,3])
        sage: M = ind(g)
        sage: M.nrows() == ind.dimension()
        True

        sage: rep_S3 = SymmetricGroupRepresentation([2,1])
        sage: ind = InducedRepresentation(4, 3, rep_S3)
        sage: ind.dimension()
        8
    """

    def __init__(self, n, k, sub_representation):
        """
        INPUT:

        - ``n`` -- integer, size of the full symmetric group
        - ``k`` -- integer, size of the subgroup (0 ≤ k ≤ n)
        - ``sub_representation`` -- representation of `S_k`
        """
        if not (0 <= k <= n):
            raise ValueError("Require 0 ≤ k ≤ n")

        self.n = n
        self.k = k
        self.sub_rep = sub_representation
        self._coset_reps = self._compute_coset_representatives()

    def _repr_(self):
        if hasattr(self.sub_rep, "_partition"):
            return (
                "Induced representation of symmetric group of "
                f"representation associated to partition {self.sub_rep._partition} "
                f"from S_{self.k} to S_{self.n}"
            )
        return f"Induced representation from S_{self.k} to S_{self.n}"

    def _compute_coset_representatives(self):
        r"""
        Compute representatives for the left cosets `S_n / S_k`.

        These correspond to ways to choose which k elements
        the subgroup acts on, which gives `\binom{n}{k}` cosets.
        """
        reps = []
        for subset in Combinations(range(1, self.n + 1), self.k):
            perm_list = list(range(1, self.n + 1))
            for i, elem in enumerate(subset):
                j = perm_list.index(elem)
                perm_list[i], perm_list[j] = perm_list[j], perm_list[i]
            reps.append(Permutation(perm_list))
        return reps

    def _sub_rep_matrix(self, perm):
        r"""Return the matrix of the `S_k` representation."""
        return self.sub_rep(perm)

    def __call__(self, g):
        r"""
        Return the induced representation matrix evaluated at ``g``.

        INPUT:

        - ``g`` -- permutation in `S_n`

        EXAMPLES::

            sage: rep_S2 = SymmetricGroupRepresentation([2])
            sage: ind = InducedRepresentation(3, 2, rep_S2)
            sage: g = Permutation([1,3,2])
            sage: M = ind(g)
            sage: M.is_square()
            True
        """
        if not isinstance(g, Permutation):
            g = Permutation(g)

        cosets = self._coset_reps
        m = len(cosets)

        d = self._sub_rep_matrix(
            Permutation(list(range(1, self.k + 1)))
        ).nrows()

        zero = matrix(QQ, d, d, 0)
        blocks = [[zero for _ in range(m)] for _ in range(m)]

        for i, r_i in enumerate(cosets):
            for j, r_j in enumerate(cosets):
                h = r_i * g * r_j.inverse()
                h_list = list(h)

                # Check if h fixes {k+1,...,n} and permutes {1,...,k}
                if sorted(h_list[:self.k]) == list(range(1, self.k + 1)):
                    h_k = Permutation(h_list[:self.k])
                    blocks[i][j] = self._sub_rep_matrix(h_k)

        return block_matrix(blocks)

    def dimension(self):
        r"""
        Return the dimension of the induced representation.

        The formula is `\dim(\mathrm{Ind}(V)) = [G:H] \cdot \dim(V)`,
        where `[G:H] = \binom{n}{k}`.

        EXAMPLES::

            sage: rep = SymmetricGroupRepresentation([1,1])
            sage: ind = InducedRepresentation(4, 2, rep)
            sage: ind.dimension()
            6
        """
        index = len(self._coset_reps)
        d = self._sub_rep_matrix(
            Permutation(list(range(1, self.k + 1)))
        ).nrows()
        return index * d


# ====================================================================
# Restricted representation
# ====================================================================

class RestrictedRepresentation(SymmetricGroupRepresentation_generic_class):
    r"""
    Restriction of a representation of `S_n` to `S_k`.

    The subgroup `S_k` is embedded as permutations that act on
    `\{1,...,k\}` and fix `\{k+1,...,n\}`.

    EXAMPLES::

        sage: rep_S4 = SymmetricGroupRepresentation([2,2])
        sage: res = RestrictedRepresentation(4, 3, rep_S4)
        sage: res.dimension()
        2

        sage: g = Permutation([2,1,3])
        sage: res(g)
        [ 1  0]
        [ 0 -1]
    """

    def __init__(self, n, k, representation):
        if k > n:
            raise ValueError("Require k ≤ n")

        self.n = n
        self.k = k
        self.rep = representation

    def _repr_(self):
        if hasattr(self.rep, "_partition"):
            return (
                "Restricted representation of symmetric group of "
                f"representation associated to partition {self.rep._partition} "
                f"from S_{self.n} to S_{self.k}"
            )
        return f"Restricted representation from S_{self.n} to S_{self.k}"

    def __call__(self, g):
        r"""
        Evaluate the restricted representation on an element of `S_k`.

        INPUT:

        - ``g`` -- permutation in `S_k`
        """
        if not isinstance(g, Permutation):
            g = Permutation(g)

        if len(g) != self.k:
            raise ValueError("Permutation must lie in S_k")

        g_ext = list(g) + list(range(self.k + 1, self.n + 1))
        return self.rep(Permutation(g_ext))

    def dimension(self):
        r"""
        Return the dimension of the restricted representation.

        EXAMPLES::

            sage: rep_S4 = SymmetricGroupRepresentation([3,1])
            sage: res = RestrictedRepresentation(4, 2, rep_S4)
            sage: res.dimension()
            3
        """
        return self(
            Permutation(list(range(1, self.k + 1)))
        ).nrows()


# ====================================================================
# Frobenius reciprocity verification
# ====================================================================

def verify_frobenius_reciprocity(n, k, rep_H, rep_G, verbose=False):
    r"""
    Verify Frobenius reciprocity for symmetric groups.

    This checks the identity:

    .. MATH::

        \langle\mathrm{Ind}_{S_k}^{S_n}(\chi), \psi\rangle_{S_n}
        = \langle\chi, \mathrm{Res}_{S_n}^{S_k}(\psi)\rangle_{S_k}

    where `\chi` is the character of ``rep_H`` (a representation of `S_k`)
    and `\psi` is the character of ``rep_G`` (a representation of `S_n`).

    EXAMPLES::

        sage: rep_S2 = SymmetricGroupRepresentation([2])
        sage: rep_S3 = SymmetricGroupRepresentation([2,1])
        sage: verify_frobenius_reciprocity(3, 2, rep_S2, rep_S3)
        True

        sage: rep_S2_sign = SymmetricGroupRepresentation([1,1])
        sage: rep_S3_triv = SymmetricGroupRepresentation([3])
        sage: verify_frobenius_reciprocity(3, 2, rep_S2_sign, rep_S3_triv)
        True
    """
    ind_rep = InducedRepresentation(n, k, rep_H)
    res_rep = RestrictedRepresentation(n, k, rep_G)

    def chi_H(g):
        return rep_H(g).trace()

    def chi_G(g):
        return rep_G(g).trace()

    def chi_ind(g):
        return ind_rep(g).trace()

    def chi_res(g):
        return res_rep(g).trace()

    def inner_product(chi1, chi2, G):
        return QQ(sum(chi1(g) * chi2(g) for g in G)) / QQ(G.cardinality())

    S_n = Permutations(n)
    S_k = Permutations(k)

    left = inner_product(chi_ind, chi_G, S_n)
    right = inner_product(chi_H, chi_res, S_k)

    if verbose:
        print(f"<Ind χ, ψ>_{{S_n}} = {left}")
        print(f"<χ, Res ψ>_{{S_k}} = {right}")
        print(f"Difference = {abs(left - right)}")

    return left == right


class ExternalTensorProductRepresentation(SymmetricGroupRepresentation_generic_class):
    r"""
    External tensor product of two symmetric group representations.

    Given representations ``rho1`` of ``S_{k1}`` and ``rho2`` of ``S_{k2}``,
    this defines a representation of ``S_{k1} x S_{k2}`` acting on the tensor
    product of the underlying vector spaces.

    The action is given by::

        (g1, g2) · (v1 ⊗ v2) = rho1(g1)v1 ⊗ rho2(g2)v2

    INPUT:

    - ``rho1`` -- a representation of ``S_{k1}``
    - ``rho2`` -- a representation of ``S_{k2}``

    EXAMPLES::
        sage: rep_S2 = SymmetricGroupRepresentation([2])
        sage: rep_S3 = SymmetricGroupRepresentation([2,1])
        sage: ext_tensor = ExternalTensorProductRepresentation(rep_S2, rep_S3)
        sage: ext_tensor.degree()
        (2, 3)
        sage: ext_tensor.dimension()
        6
        sage: g1 = Permutation([2,1])
        sage: g2 = Permutation([2,3,1])
        sage: M = ext_tensor((g1, g2))
        sage: M.nrows() == ext_tensor.dimension()
        True
    """

    def __init__(self, rho1, rho2):
        self._rho1 = rho1
        self._rho2 = rho2

        la1 = rho1._partition
        la2 = rho2._partition
        self._partitions = (la1, la2) if la1 and la2 else None

        self.n = rho1._n + rho2._n

    def _repr_(self):
        return "External tensor product of {} and {}".format(
            self._rho1, self._rho2
        )

    def degree(self):
        """
        Return the pair of degrees (k1, k2).
        """
        return (self._rho1._n, self._rho2._n)

    def dimension(self):
        """
        Return the dimension of the tensor product space.
        """
        dim1 = Permutations(self._rho1._n).algebra(QQ).specht_module(self._rho1._partition).dimension()
        dim2 = Permutations(self._rho2._n).algebra(QQ).specht_module(self._rho2._partition).dimension()
        return dim1 * dim2

    def representation_matrix(self, g):
        r"""
        Return the representation matrix of ``g = (g1, g2)``.

        INPUT:

        - ``g`` -- a tuple ``(g1, g2)`` with ``g1 ∈ S_{k1}``, ``g2 ∈ S_{k2}``
        """
        try:
            g1, g2 = g
        except (TypeError, ValueError):
            raise TypeError(
                "Group element must be a pair (g1, g2)"
            )

        M1 = self._rho1(g1)
        M2 = self._rho2(g2)

        return M1.tensor_product(M2)
    
    def __call__(self, g):
        r"""
        Return the representation matrix of ``g = (g1, g2)``.

        INPUT:

        - ``g`` -- a tuple ``(g1, g2)`` with ``g1 ∈ S_{k1}``, ``g2 ∈ S_{k2}``
        """
        return self.representation_matrix(g)

In [12]:
rep_S2 = SymmetricGroupRepresentation([2])
ind = InducedRepresentation(3, 2, rep_S2)
print("dim of induced rep'n from trivial 2 dim'l rep'n to S_3: ", ind.dimension())

g = Permutation([2,1,3])
M = ind(g)
print("M.nrows() == ind.dimension(): ", M.nrows() == ind.dimension())


rep_S3 = SymmetricGroupRepresentation([2,1])
ind = InducedRepresentation(4, 3, rep_S3)
print("dim of induced rep'n from [2,1] dim'l rep'n of S_3 to S_4: ", ind.dimension())

dim of induced rep'n from trivial 2 dim'l rep'n to S_3:  3
M.nrows() == ind.dimension():  True
dim of induced rep'n from [2,1] dim'l rep'n of S_3 to S_4:  8


In [13]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
print(sym_repn_3(Permutation([1,2,3])))

[1 0]
[0 1]


In [14]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
ind_rep_4_3_sym21 = InducedRepresentation(4,3,sym_repn_3)
print(ind_rep_4_3_sym21)

Induced representation of symmetric group of representation associated to partition [2, 1] from S_3 to S_4


In [ ]:
# Trivial representation of S_2
def trivial_rep_s2(perm):
    return matrix(ZZ, 1, 1, [1])

# Create induced representation
ind_rep = InducedRepresentation(3, 2, trivial_rep_s2)

In [16]:
ind_rep.dimension()

3

In [17]:
ind_rep(Permutation([1,2,3]))

[1|0|0]
[-+-+-]
[0|1|0]
[-+-+-]
[0|0|1]

In [19]:
mat_321 = ind_rep_4_3_sym21(Permutation([3,2,1])); mat_321

[-1  1| 0  0| 0  0| 0  0]
[ 0  1| 0  0| 0  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0  0| 0 -1]
[ 0  0| 0  0| 0  0|-1  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0 -1| 0  0]
[ 0  0| 0  0|-1  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0 -1| 0  0| 0  0]
[ 0  0|-1  0| 0  0| 0  0]

In [20]:
mat_321*mat_321

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

In [21]:
rep = SymmetricGroupRepresentation([1,1])
ind = InducedRepresentation(4, 2, rep)
ind.dimension()

6

In [ ]:
rep_S4 = SymmetricGroupRepresentation([2,2])
res = RestrictedRepresentation(4, 3, rep_S4)
res.dimension()


2

In [24]:
g = Permutation([2,1,3])
res(g)

[ 1  0]
[ 1 -1]

In [87]:
rep_S2 = SymmetricGroupRepresentation([2])
rep_S3 = SymmetricGroupRepresentation([2,1])
ext_tensor = ExternalTensorProductRepresentation(rep_S2, rep_S3)
g1 = Permutation([2,1])
g2 = Permutation([2,3,1])
M = ext_tensor((g1, g2))
print(M)
print(ext_tensor.degree())
print(ext_tensor.dimension())
print(M.nrows() == ext_tensor.dimension())

[-1  1]
[-1  0]
(2, 3)
2
True
